### GPT загвар Монгол хэлэн дээр уридчилч (Pretrain) сургах

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ghOi5FCs8bGQGLzRNFQkmw4v0MUwyiyJ?usp=sharing)

<div class="align-center">
  <a href="https://www.youtube.com/@deeplearning_mn"><img src="https://www.svgrepo.com/show/13671/youtube.svg" height="50"></a>
     <a href="https://www.facebook.com/groups/1026243374478030"><img src="https://www.svgrepo.com/show/475647/facebook-color.svg" height="45"></a>
 <a href="https://docs.google.com/forms/d/e/1FAIpQLSetEXXZP89FrkST-ox4PDRfDCvbResPN3FKoNmJJkq_hKv8Qw/viewform"><img src="https://www.svgrepo.com/show/324465/neuron-connections-neural-network.svg" height="50"></a>


 ⭐️ <i>Star us on <a href="https://github.com/deeplearningmn/AIEng">Github</a> </i> ⭐️
</div>

## Own GPT

In [1]:
#Сангууд, багц сангууд
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

#Өгөгдөл боловсруулах

In [2]:
#Data
with open('/content/dataset_naagii.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [3]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

##Bigram model гэж юу вэ?
Bigram model гэдэг нь нэг үгийг дагаж ирэх дараагийн үгийн магадлалыг тооцдог хэлний загвар юм. Товчоор хэлбэл, энэ нь үгсийн дарааллын хоёр үгт хос дээр үндэслэн тухайн дараагийн үгийг таамагладаг.

Текст: "Намайг ЧатЖПТ гэдэг"

Bigram-ууд:
1. Намайг → ЧатЖПТ
2. ЧатЖПТ → гэдэг

#Загвар

In [5]:

# hyperparameters
batch_size = 32#16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 8000 #5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128#64
n_head = 4
n_layer = 4
dropout = 0.1
# ------------

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# simple bigram model

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx




#Сургах

In [6]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



0.809268 M parameters
step 0: train loss 4.0732, val loss 4.0914
step 100: train loss 1.8247, val loss 2.0712
step 200: train loss 1.0901, val loss 2.0757
step 300: train loss 0.5272, val loss 2.3519
step 400: train loss 0.3140, val loss 2.8171
step 500: train loss 0.2422, val loss 3.0642
step 600: train loss 0.2169, val loss 3.3050
step 700: train loss 0.1993, val loss 3.4594
step 800: train loss 0.1943, val loss 3.7065
step 900: train loss 0.1886, val loss 3.6788
step 1000: train loss 0.1831, val loss 3.7305
step 1100: train loss 0.1797, val loss 3.8686
step 1200: train loss 0.1788, val loss 3.7290
step 1300: train loss 0.1774, val loss 3.9076
step 1400: train loss 0.1721, val loss 4.0000
step 1500: train loss 0.1699, val loss 3.9741
step 1600: train loss 0.1714, val loss 4.0190
step 1700: train loss 0.1713, val loss 4.2507
step 1800: train loss 0.1701, val loss 4.0259
step 1900: train loss 0.1668, val loss 4.0992
step 2000: train loss 0.1670, val loss 4.1658
step 2100: train loss 0.

In [8]:
# generate from the model
def generate():
  context = torch.zeros((1, 1), dtype=torch.long, device=device)
  print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

# generate from the model
generate()


Нарийн өвс ургасан малын сайхан бэлчээртэй
Нааш цааш сүлжих тэнэгэр сайхан нутагтай
Дөрвөн цагийн улиралд дураараа нүүх нутагтай
Таван зүйлийн тарианы хөрс шороо газартай

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Өвлийн тэсгим цагт цасан мөсөн бvрхээд
Болор шилийн өнгө гялтганаж, гялалзсан орон
Зуны найртай улиралд цэцэг, навч дэлгэрээд
Жигүүртэн шувуу холоос ирж га газар
Гөрөөс араатан авласан урт урт шилнvvд
Хүлэг морьд уралдсан хөндий сайхан хоолойнууд

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Өвлийн тэсгим цагт цасан мөсөн бvрхээд
Болор шилийн өнгө гял, Увс, Буйрын гүн цэнхэр нуурууд
Хүн малын ундаа болсон тойром бүрд уснууд

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Орхон, говийн хооронд Халхын уудам нутаг
Хар бага наснаас хөндлөн гулд давхисан газар
Гөрөөс араатан авласан урт урт шилнvvд
Хүлэг морьд уралдсан хөндий сайхан хоолойнууд

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Хүннү, СүннүгийнСайн vеэс хөгшид өвгөдийн минь нутаг
Хөх Монголын цагт хүчи

In [ ]:
# generate again from the model
generate()


Нарийн өвс ургасан малын сайхан бэлчээртэй
Нам цагийн улиралд дураараа нүүх нутагтай
Таван зүйлийн тарианы хөрс шороо газартай

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Нарийн өвс ургасан малын сайхан бэлчээрхээд
Болол миний төрсөн нутаг
Монголын сайхан орон

Хүннү, Сүннүгийн vеэс хөгшид өвгөдийн минь нутаг
Хөх Монголын цагт хүчирхэг болсон орон
Он онд идээшиж олон жилд дассан нутаг
Одоогийн шинэ Монголын улаан туг бvрхсэн нутаг

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Салхины үзүүрт найгасан соргог нарийн ногоотой
Саруул талд яралзсан сонин янзын зэрэглээтэй
Сайн эрс цугласан байц бэрх газарийн ногоотой
Саруул талд яралзсан сонин янзын нзэрс цугасан байц бэрх газартай
Сүлд тахилга улирсан сvмбэр их овоотой

Энэ бол миний төрсөн нутаг
Монголын сайхан орон

Өссөн төрсөн бидний үндэсний хайртай нутаг
Өнгөлзөгч дайсан ирвэл үтэр өшиглөн өвгөдийг тавьсан газар
Үр, ач, хүүхдийн үржиж өссөн газар
Таван хошуу мал тал дүүрэн бэлчсэн нутаг
Монгоунгол хvн бидний сэтгэлийг сорсо